### Example 1

In this example, I will just use a normal json file with one level nested keys

In [1]:
import utils.flatten_json as fj

In [2]:
data = [{'state': 'Florida',
          'shortname': 'FL',
          'info': {
               'governor': 'Rick Scott'
          },
          'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
         {'state': 'Ohio',
          'shortname': 'OH',
          'info': {
               'governor': 'John Kasich'
          },
          'counties': [{'name': 'Summit', 'population': 1234},
                       {'name': 'Cuyahoga', 'population': 1337},
                      {'name': 'Cuyahoga2', 'population': 1331}]}]

In [3]:
df = fj.flatten_json(data)
df

,state,shortname,info.governor,counties.name,counties.population
0,Florida,FL,Rick Scott,"[Dade, Broward, Palm Beach]","[12345, 40000, 60000]"
1,Ohio,OH,John Kasich,"[Summit, Cuyahoga, Cuyahoga2]","[1234, 1337, 1331]"


### Example 2

In [4]:
import utils.flatten_json as fj
data = [{"isbn": "123-456-222",  
 "author": 
    {
      "lastname": "Doe",
      "firstname": "Jane"
    },
"editor": 
    {
      "lastname": "Smith",
      "firstname": "Jane"
    },
  "title": "The Ultimate Database Study Guide",  
  "category": ["Non-Fiction", "Technology"]
 }]



In [5]:
df = fj.flatten_json(data)
df

,isbn,author.lastname,author.firstname,editor.lastname,editor.firstname,title,category
0,123-456-222,Doe,Jane,Smith,Jane,The Ultimate Database Study Guide,"[Non-Fiction, Technology]"


### Example 3

In [7]:
import utils.flatten_json as fj
data = [{
"problems": [{
    "Diabetes":[{
        "medications":[{
            "medicationsClasses":[{
                "className":[{
                    "associatedDrug":[{
                        "name":"asprin",
                        "dose":"",
                        "strength":"500 mg"
                    }],
                    "associatedDrug#2":[{
                        "name":"somethingElse",
                        "dose":"",
                        "strength":"500 mg"
                    }]
                }],
                "className2":[{
                    "associatedDrug":[{
                        "name":"asprin",
                        "dose":"",
                        "strength":"500 mg"
                    }],
                    "associatedDrug#2":[{
                        "name":"somethingElse",
                        "dose":"",
                        "strength":"500 mg"
                    }]
                }]
            }]
        }],
        "labs":[{
            "missing_field": "missing_value"
        }]
    }],
    "Asthma":[{}]
}]}]

In [8]:
df = fj.flatten_json(data)
df

,problems.Diabetes.medications.medicationsClasses.className.associatedDrug.name,problems.Diabetes.medications.medicationsClasses.className.associatedDrug.dose,problems.Diabetes.medications.medicationsClasses.className.associatedDrug.strength,problems.Diabetes.medications.medicationsClasses.className.associatedDrug#2.name,problems.Diabetes.medications.medicationsClasses.className.associatedDrug#2.dose,problems.Diabetes.medications.medicationsClasses.className.associatedDrug#2.strength,problems.Diabetes.medications.medicationsClasses.className2.associatedDrug.name,problems.Diabetes.medications.medicationsClasses.className2.associatedDrug.dose,problems.Diabetes.medications.medicationsClasses.className2.associatedDrug.strength,problems.Diabetes.medications.medicationsClasses.className2.associatedDrug#2.name,problems.Diabetes.medications.medicationsClasses.className2.associatedDrug#2.dose,problems.Diabetes.medications.medicationsClasses.className2.associatedDrug#2.strength,problems.Diabetes.labs.missing_field
0,asprin,,500 mg,somethingElse,,500 mg,asprin,,500 mg,somethingElse,,500 mg,missing_value


### Example 4

In [9]:
import utils.flatten_json as fj
data = [{ 
  "accounting" : [   
                     { "firstName" : "John",  
                       "lastName"  : "Doe",
                       "age"       : 23 },

                     { "firstName" : "Mary",  
                       "lastName"  : "Smith",
                        "age"      : 32 }
                 ],                            
  "sales"      : [ 
                     { "firstName" : "Sally", 
                       "lastName"  : "Green",
                        "age"      : 27 },

                     { "firstName" : "Jim",   
                       "lastName"  : "Galley",
                       "age"       : 41 }
                 ] 
} ]

In [10]:
df = fj.flatten_json(data)
df

,accounting.firstName,accounting.lastName,accounting.age,sales.firstName,sales.lastName,sales.age
0,"[John, Mary]","[Doe, Smith]","[23, 32]","[Sally, Jim]","[Green, Galley]","[27, 41]"


**All the JSON examples that I have used here are a copied from google, json.org, and stackoverflow. I do not own these examples and give credits to the creators of these examples**